# Fine-tuning Mistral-7b AI using Peft QLoRa (WIP)

Note that this could be used for any model that supports device_map (i.e. loading the model with accelerate).

This notebook demonstrates how to finetune Mistral-7B with peft, bnb, SFTT:


*   **Step 0**: Initialise the project
*   **Step 1**: Load the base and quantized model and run local inference
*   **Step 2**: Finetune the model with LoRA
*   **Step 3**: Push the LoRA weights to Hugging Face Hub (or GCS)
*   **Step 4**: Evaluate the model and run inference
*   **[Optional]**: Merge the LORA adapter to the base model, deploy with vLLM

# Step 0 -   Initialise the project

#### Define some variables and APIs

In [1]:
# Cloud project id.
PROJECT_ID = "[PROJECT_ID]"  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = "[REGION]"  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output.
# Start with gs:// prefix, e.g. gs://foo_bucket.
BUCKET_URI = "[BUCKET_URI]"  # @param {type:"string"}

! gcloud config set project $PROJECT_ID

import os

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")

# The service account looks like:
# '@.iam.gserviceaccount.com'
# Please go to https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# and create service account with `Vertex AI User` and `Storage Object Admin` roles.
# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = "[SERVICE_ACCOUNT]"  # @param {type:"string"}

Updated property [core/project].


#### Initialize Vertex AI API


In [2]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

#### Define constants

In [3]:
VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20231127_0916_RC00"
TRAIN_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-peft-train:20231205_0936_RC00"

#### Define Common Functions

Define a wrapper function which pass our query to the model for inference and return decoded model's completion(response).

In [4]:
from datetime import datetime

def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Question:
  {query}

  ### Answer:
  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=250, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids)
  return (decoded[0])

def create_name_with_datetime(prefix: str) -> str:
    """Creates a name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


Define model deployment functions

In [5]:
def deploy_model_vllm(
    model_name,
    model_id,
    service_account,
    machine_type="g2-standard-8",
    accelerator_type="NVIDIA_L4",
    accelerator_count=1,
):
    """Deploys trained models with vLLM into Vertex AI."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")

    dtype = "bfloat16"
    if accelerator_type in ["NVIDIA_TESLA_T4", "NVIDIA_TESLA_V100"]:
        dtype = "float16"

    vllm_args = [
        "--host=0.0.0.0",
        "--port=7080",
        f"--model={model_id}",
        f"--tensor-parallel-size={accelerator_count}",
        "--swap-space=16",
        f"--dtype={dtype}",
        "--gpu-memory-utilization=0.9",
        "--disable-log-stats",
    ]
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=VLLM_DOCKER_URI,
        serving_container_command=["python", "-m", "vllm.entrypoints.api_server"],
        serving_container_args=vllm_args,
        serving_container_ports=[7080],
        serving_container_predict_route="/generate",
        serving_container_health_route="/ping",
    )

    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )
    return model, endpoint

## Install necessary packages
First, install the dependencies below to get started. As these features are available on the main branches only, we need to install the libraries below from source.

In [6]:
# Using BitsAndBytes Library for quantization
!pip install -q -U bitsandbytes

# Tranformers provides all API for downloading and working with pre-trained models that are in the HF hub.
!pip install -q -U git+https://github.com/huggingface/transformers.git

# This package provides all the APIs we will need to perform the LoRA technique.
!pip install -q -U git+https://github.com/huggingface/peft.git

# Powerful huggingface package, that hides the complexity of the developer trying to write/manage code needed to use multi-GPUs/TPU/fp16.
!pip install -q -U git+https://github.com/huggingface/accelerate.git

# This huggingface package provides access to the various datasets in the huggingface hub.
!pip install -q datasets

# This library provides access to the Weights and Biases library to capture various metrics, during the fine-tuning process.
!pip install -q wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

# Step 1 - Model loading and local inference
We'll load the model using QLoRA quantization to reduce the usage of memory


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # loading the base model in 4bit quantization. Also need to check model weights in config file.
    bnb_4bit_use_double_quant=True, # Double Quantization
    bnb_4bit_quant_type="nf4", # vs FP4 ? Read Paper
    bnb_4bit_compute_dtype=torch.bfloat16 # Would use float16 with compute capabilities below 8 (T4, V100)
)

Now we specify the model ID and then we load it with our previously defined quantization configuration.

In [8]:
model_id = "mistralai/Mistral-7B-v0.1"

# Load Mistral-7B quantized with BitsAndBytesConfig defined above.

# model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"": 0})

# Define the tokenizer
# Using AutoTokenizers for creating a tokenizer for Mistral-7B
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/966 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Run a inference on the base model. The model does not seem to understand our instruction and gives us a list of questions related to our query.

In [9]:
result = get_completion(query="What is Model Garden?", model=model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s> 
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Question:
  What is Model Garden?

  ### Answer:
  </s> 3703 ë³† 5 ³„í†‚ ë¡œë‹ˆë¬ ³ª ì‡ˆ ìœˆ ì„
  ë¡œë‹ˆë¬ ì„œ 3703 ì„ ë¡œ 2004âˆ’2010
  ì„œë¡œë¡œ ìƒ˜ìƒ˜ êµ˜ í™” 10 ì¤œ ìˆ˜ì‚œ

  í™” 10 ì„œíŽ‹ ìŠµê²” í•Œ ì¼ì–´ ì•ˆíŽœê¸°

  ### [Maps]
  ë¡œë‹ˆë¬ ì„œ 2004âˆ’2015 ë¡œ 3700.0 ë¡œë˜¸ ìœ¨ì˜˜ ë¡œ569.00 ìˆ˜ì‹œ 3680.0 ô×


# Step 2 - Finetune the model with LoRA

## Load dataset for finetuning

Let's load a dataset on finance, to fine tune our model on basic finance knowledges. In this guide, we'll load 10% data from the original dataset for the sake of the demo just to showcase how to use this integration with existing tools on the HF ecosystem.

In [10]:
from datasets import load_dataset

data = load_dataset("fredmo/vertexai-qna-500", split='train') #Full train split

# Explore the data
df = data.to_pandas()
df.head(10)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

,input_text,output_text
0,question: What is Vertex AI?,Vertex AI is a unified machine learning platfo...
1,question: What is Vertex AI?,Vertex AI makes it easy for businesses to buil...
2,question: What are some use cases for Vertex AI?,Vertex AI can be used for a wide variety of us...
3,question: What is Vertex AI?,Vertex AI is a unified machine learning platfo...
4,question: What are the different options for m...,Vertex AI offers several options for model tra...
5,question: What are the MLOps tools available i...,Vertex AI provides a range of MLOps tools for ...
6,question: What is the difference between PaLM ...,PaLM API for text is fine-tuned for language t...
7,question: What is model tuning?,Model tuning is the process of customizing the...
8,question: What is streaming?,Streaming is a feature that allows you to rece...
9,question: What are the different ways to acces...,"You can access the API via REST, gRPC, or one ..."


Instruction Fintuning - Prepare the dataset under the format of "prompt" so the model can better understand :
1. the function generate_prompt : take the instruction and output and generate a prompt
2. shuffle the dataset
3. tokenizer the dataset

In [11]:
def generate_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """
    text = 'Below is an instruction that describes a question. Write a response that ' \
            'appropriately answer the request.\n\n'
    text += f'### Instruction:\n{data_point["input_text"]}\n\n'
    text += f'### Response:\n{data_point["output_text"]}'
    return text

# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in data]
data = data.add_column("prompt", text_column)

We'll need to tokenize our data so the model can understand.


In [12]:
data = data.shuffle(seed=1234)  # Shuffle dataset here
data = data.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/531 [00:00<?, ? examples/s]

Split dataset into 90% for training and 10% for testing

In [13]:
data = data.train_test_split(test_size=0.1)
train_data = data["train"]
test_data = data["test"]

In [14]:
print(test_data)

Dataset({
    features: ['input_text', 'output_text', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 54
})


## Apply Lora  
Here comes the magic with peft! Let's load a PeftModel and specify that we are going to use low-rank adapters (LoRA) using get_peft_model utility function and  the prepare_model_for_kbit_training method from PEFT.

In [15]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable() # Discarding intermediate activation values during the forward pass, add computation in backward pass
model = prepare_model_for_kbit_training(model) # TODO: Explain

In [16]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
   

Use the following function to find out the linear layers for fine tuning.
QLoRA paper : "We find that the most critical LoRA hyperparameter is how many LoRA adapters are used in total and that LoRA on all linear transformer block layers is required to match full finetuning performance."

In [17]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [18]:
modules = find_all_linear_names(model)
print(modules)

['down_proj', 'q_proj', 'up_proj', 'k_proj', 'o_proj', 'gate_proj', 'v_proj']


In [19]:
from peft import LoraConfig, get_peft_model

# PEFT library supports various other PEFT methods such as prefix tuning, P-tuning, and Prompt Tuning. etc.
# Since we are using the LoRA method, we are using the LoraConfig class.

lora_config = LoraConfig(
    r=8, # dimension of the low-rank matrix
    lora_alpha=32, # adjusts the magnitude of the combined result (base model output + low-rank adaptation)
    target_modules=modules, # TODO: Describe ; Optional
    lora_dropout=0.05, # 5% dropout neuron probability of the LoRA layers. To avoid overfitting.
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [20]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 20971520 | total: 7262703616 | Percentage: 0.2888%


## Run the training!

In [21]:
import wandb

wandb.login()
%env WANDB_PROJECT=python-fine-tuning

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=python-fine-tuning


Setting the training arguments:
* for the reason of demo, we just ran it for few steps (100) just to showcase how to use this integration with existing tools on the HF ecosystem.

In [22]:
import transformers

import locale
locale.getpreferredencoding = lambda: "UTF-8"

# “Transformer Reinforcement Learning” is used for fine-tuning the transformer model using reinforcement learning.
# We will use our instruction dataset to perform this reinforcement learning and fine-tune the model.
# We will be using SFTrainer object to perform the fine-tuning.

!pip install -q trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 9.9 MB/s eta 0:00:00


In [23]:
# Some parameters to consider:
# gradient_checkpointing (already enabled on the model). Used to reduce mem by re-computing intermediate activations during backwards instead of storing them all.
# weigth decay : to prevent overfitting by adding penalty to loss function

trainingArgs = transformers.TrainingArguments(
        per_device_train_batch_size=3, # Batch size per GPU
        gradient_accumulation_steps=4, # Number of update steps to accumulate the gradient for
        warmup_steps=0.03,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1, # Frequency of logging
        output_dir="outputs", # Model predictions and checkpoints storage
        optim="paged_adamw_8bit", # optimizer is responsible for computing the gradient statistics for back propagation. Done in 8-bit to save memory.
        report_to="wandb",
        save_strategy="epoch", # save after every epoch
    )

In [24]:
from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token #TODO: Explain
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data, #TODO: Explain
    dataset_text_field="prompt", #TODO: Explain
    peft_config=lora_config,
    args=trainingArgs,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False), #TODO: CLM and not MLM
)

#MLM is a training method used in models like BERT, where some tokens in the input sequence are masked,
# and the model learns to predict the masked tokens based on the surrounding context.
# MLM has the advantage of bidirectional context, allowing the model to consider both past and future tokens when making predictions.
# This approach is especially useful for tasks like text classification, sentiment analysis, and named entity recognition.



/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/477 [00:00<?, ? examples/s]

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:247: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


Start the training

In [25]:
print('Start the supervised fine tuning of Mistral-7B')

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

print('Done Training')

Start the supervised fine tuning of Mistral-7B


wandb: Currently logged in as: thomas-lemoullec. Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,2.691000
2,2.411500
3,1.918400
4,1.341200
5,1.184900
6,0.978300
7,1.098500
8,1.074400
9,1.133700
10,1.139900


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Done Training


In [26]:
#stop reporting to wandb
wandb.finish()

# save model
trainer.save_model()
print("Model saved")

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▆▃▃▄▃▃▃▃▂▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▁▁▂▂▂▂▂▂▂▁▂▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,2.52
train/global_step,100


Model saved


# Step 3 - Push the finetuned Lora Adapter to HF Hub (or GCS)

In [28]:
from huggingface_hub import notebook_login

# Log in to HF Hub
notebook_login()

In [29]:
# push to hub the LORA adapter
model.push_to_hub("Thomas-lemoullec/mistral_7b_vertexQandA")
tokenizer.push_to_hub("Thomas-lemoullec/mistral_7b_vertexQandA")

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Thomas-lemoullec/mistral_7b_vertexQandA/commit/a95d0a31c2045f9415e3d6c0a1500e3c08726e5e', commit_message='Upload tokenizer', commit_description='', oid='a95d0a31c2045f9415e3d6c0a1500e3c08726e5e', pr_url=None, pr_revision=None, pr_num=None)

In [30]:
import torch
import torch.nn as nn
import gc
# clear the VRAM


def memory_stats():
    print('allocated:')
    print(torch.cuda.memory_allocated()/1024**2)
    print('cached:')
    print(torch.cuda.memory_cached()/1024**2)

memory_stats()

#del trained_model
#del lora_merged_model
#del trainer
#del model
#del tokenizer

torch.cuda.empty_cache()
gc.collect()
memory_stats()

allocated:
5070.0654296875
cached:
8042.0


/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:444: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


allocated:
5070.0654296875
cached:
5384.0


# Step 4 - Evaluate the model and run inference



In [31]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Load directly adapters from the Hub using the command below

In [32]:
# Based on your business need, if you need the base model and the finetuned LoRA weight to be served separately

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "Thomas-lemoullec/mistral_7b_vertexQandA"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_4bit=True, device_map={"": 0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

You can then directly use the trained model that you have loaded from the 🤗 Hub for inference as you would do it usually in transformers.

In [33]:
result = get_completion(query="What is Model Garden?", model=model, tokenizer=tokenizer)
print(result)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


<s> 
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Question:
  What is Model Garden?

  ### Answer:
   Model Garden is a platform that helps machine learning engineers and data scientists develop, test, and deploy machine learning models. It provides tools for creating and versioning models, as well as for tracking the performance of models over time.
  ### Question:
  What are the benefits of using Model Garden?  Model Garden enables you to develop, test, and deploy machine learning models more efficiently. It also makes it easy to track the performance of models over time.
  ### Question:
  What are the limitations of using Model Garden?  Model Garden has some limitations. For example, it doesn't support all types of machine learning models, and it doesn't have all of the features that some other platforms have.</s>


# [Optional] Merge the LORA adapter to the base model, deploy with vLLM

In [36]:
merge_job_name = create_name_with_datetime(prefix="mistral-peft-merge")

# The base model to be merged upon. It can be a huggingface model id, or a GCS
# path where the base model was stored.
base_model_dir = "mistralai/Mistral-7B-v0.1"  # @param {type:"string"}
# The previously trained LoRA adapter. It needs to be stored in a GCS path.
finetuned_lora_adapter_dir = "Thomas-lemoullec/mistral_7b_vertexQandA"#"gs://mistral-lora-weights/outputs"


print(finetuned_lora_adapter_dir)

# The GCS path to save the merged model
merged_model_output_dir = os.path.join(BUCKET_URI, merge_job_name)
merged_model_output_dir_gcsfuse = merged_model_output_dir.replace("gs://", "/gcs/")

machine_type = "n1-highmem-16"
accelerator_type = "NVIDIA_TESLA_V100"
accelerator_count = 2

#machine_type = "g2-standard-8"
#accelerator_type = "NVIDIA_L4"

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": machine_type,
            "accelerator_type": accelerator_type,
            "accelerator_count": accelerator_count,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": TRAIN_DOCKER_URI,
            "command": [],
            "args": [
                "--task=merge-causal-language-model-lora",
                "--merge_model_precision_mode=float16",
                "--pretrained_model_id=%s" % base_model_dir,
                "--finetuned_lora_model_dir=%s" % finetuned_lora_adapter_dir,
                "--merge_base_and_lora_output_dir=%s" % merged_model_output_dir_gcsfuse,
            ],
        },
    }
]

merge_custom_job = aiplatform.CustomJob(
    display_name=merge_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

merge_custom_job.run()

print("The merged model is stored at: ", merged_model_output_dir)

INFO:google.cloud.aiplatform.jobs:Creating CustomJob


Thomas-lemoullec/mistral_7b_vertexQandA


INFO:google.cloud.aiplatform.jobs:CustomJob created. Resource name: projects/314837540096/locations/europe-west4/customJobs/5546499050699751424
INFO:google.cloud.aiplatform.jobs:To use this CustomJob in another session:
INFO:google.cloud.aiplatform.jobs:custom_job = aiplatform.CustomJob.get('projects/314837540096/locations/europe-west4/customJobs/5546499050699751424')
INFO:google.cloud.aiplatform.jobs:View Custom Job:
https://console.cloud.google.com/ai/platform/locations/europe-west4/training/5546499050699751424?project=314837540096
INFO:google.cloud.aiplatform.jobs:CustomJob projects/314837540096/locations/europe-west4/customJobs/5546499050699751424 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/314837540096/locations/europe-west4/customJobs/5546499050699751424 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/314837540096/locations/europe-west4/customJobs/5546499050699751424 current state:
J

The merged model is stored at:  gs://mistral-experiment/mistral-peft-merge_20231206_174948


In [37]:
# Deploy Merged model on VLLM

model_with_peft_vllm, endpoint_with_peft_vllm = deploy_model_vllm(
    model_name=create_name_with_datetime(prefix="mistral-peft-serve-vllm"),
    model_id=merged_model_output_dir,
    service_account=SERVICE_ACCOUNT,
    machine_type="n1-highmem-16",
    accelerator_type="NVIDIA_TESLA_V100",
    accelerator_count=2,
)

print("endpoint_name:", endpoint_with_peft_vllm.name)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/314837540096/locations/europe-west4/endpoints/4229975163682160640/operations/8767304784135847936
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/314837540096/locations/europe-west4/endpoints/4229975163682160640
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/314837540096/locations/europe-west4/endpoints/4229975163682160640')
INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/314837540096/locations/europe-west4/models/4265652116980236288/operations/7091965722754023424
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/314837540096/locations/europe-west4/models/4265652116980236288@1
INFO:google.cloud.aiplatform.models:To use this Model

endpoint_name: 4229975163682160640


In [38]:
instance = {
    "prompt": "What is Model Garden?",
    "n": 1,
    "max_tokens": 250,
}
response = endpoint_with_peft_vllm.predict(instances=[instance])
print(response.predictions[0])

Prompt:
What is Model Garden?
Output:
 Model Garden is a platform that lets you add, manage, and deploy models to a Vertex AI model registry. Depending on your access level, you can also title your model and view its details page. What are the different ways that I can provide feedback on Model Garden? You can provide feedback by rating a model, leaving a comment, or suggesting an improvement. How do I leave a comment on a model in Model Garden? To leave a comment on a model in Model Garden, go to the model's details page and click the Comments tab. In the Comment on this model section, enter your comment and click Submit. How do I suggest an improvement for a model in Model Garden? To suggest an improvement for a model in Model Garden, go to the model's details page and click the Improve this model section. In the Improve this model section, enter your suggestion and click Submit. How can I vote on model suggestions in Model Garden? To vote on model suggestions in Model Garden, go to 